## Data exploration

In [ ]:
import pandas as pd
# import plotly 
import os
import pyarrow as pa
import numpy as np
import pandas as pd
import csv

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../..")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

In [ ]:
hate_df = pd.read_csv("data/toraman22_hate_speech_v2/Toraman22_hate_speech_v2.tsv", sep="\t")

indexes_to_drop = []
for index, row in hate_df.iterrows():
    try:
        favourite_int = int(row["tweet_id"])
    except:
        indexes_to_drop.append(index)

print(hate_df.shape)
print(f"Found {len(indexes_to_drop)} rows to drop")
hate_df = hate_df.drop(indexes_to_drop)
# enforce int type on new column 
# hate_df["tweet_id"] = hate_df["tweet_id"].astype(np.int64)
hate_df["tweet_id"]
print(hate_df.shape)

In [ ]:
dataset_v2 = pd.read_csv("data\hatespeech_v2\hate_speech_dataset_v2.csv")
# dataset_v2_tweet_ids = set(.values)

out_df = hate_df[hate_df["tweet_id"].isin(dataset_v2["TweetID"])]

print(out_df.shape)

In [ ]:
hate_df["tweet_id"].unique()

In [ ]:

out_df = hate_df[hate_df["tweet_id"].isin(dataset_v2_tweet_ids)]
out_df[out_df["language"] == 1].shape

In [ ]:
def prepare_hatespeech_v2_dataset():
    hate_df = pd.read_csv("data/toraman22_hate_speech_v2/Toraman22_hate_speech_v2.tsv", sep='\t')
    # hate_df.to_csv("./Toraman22_hate_speech_v2.csv", index=False)
    hate_df["tweet_id"] = hate_df["tweet_id"].str.strip()   # .str.replace(" ", "") , dtype={"tweet_id": np.int64}

    counter = 0
    for j,i in enumerate(hate_df["tweet_id"]):
        try:
            if i != np.nan:
                int(i)
        except:
            print(j, i)
            counter += 1
    print("counter", counter)
    # , dtype={"tweet_id": np.int64}

    # split into training and test
    # , dtype={"TweetID": np.int64}
    dataset_v2 = pd.read_csv("data\hatespeech_v2\hate_speech_dataset_v2_labeler.csv", dtype={"tweet_id": np.int64})
    dataset_v2_tweet_ids = dataset_v2["TweetID"].values.flatten()
    print("dataset v2", len(set(dataset_v2_tweet_ids)))

    # for i in dataset_v2_tweet_ids:
    #     if i not in hate_df["tweet_id"]:
    #         print(i)
    # keep only version 2 tweets
    # hate_df = hate_df[hate_df["tweet_id"].isin(dataset_v2_tweet_ids)]

    # hate_df["language"].unique()
    # only keep tweets where the language is english
    # hate_df = hate_df[hate_df["language"] == 1]

    #  train_df = hate_df[hate_df["split"] == "train"]
    # hate_df = hate_df[["tweet_id", "text", "label", "topic", "user_id"]]
    print(len(hate_df.index))
    return hate_df
prepare_hatespeech_v2_dataset()